In [ ]:
import pandas as pd
import pysolr


In [ ]:
# url = "http://b2find.eudat.eu/solr/select"
url = "http://localhost:8983/solr/ckan"

solr = pysolr.Solr(url)

In [ ]:
# Solr temporal coverage begin
facet = "extras_TemporalCoverageBeginDate"
solr_params = {
    'echoParams': 'none',
    'rows': 1,
    'wt': 'json',
    'q': '*',
     'fq': f"{facet}:[* TO *]",
     'fl': facet,
    'sort': f"{facet} asc",
    'indent': 'false',
}
results = solr.search(**solr_params)
start = results.docs[0][facet]
start

In [ ]:
# Solr temporal coverage end
facet = "extras_TemporalCoverageEndDate"
solr_params = {
    'echoParams': 'none',
    'rows': 1,
    'wt': 'json',
    'q': '*',
     'fq': f"{facet}:[* TO *]",
     'fl': facet,
    'sort': f"{facet} desc",
    'indent': 'false',
}
results = solr.search(**solr_params)
end = results.docs[0][facet]
end

## Solr date range field

https://solr.apache.org/guide/6_6/working-with-dates.html

In [ ]:
facet = "extras_TempCoverage"

solr_params = {
    'echoParams': 'none',
    'rows': 0,
    'wt': 'json',
    'q': '*',
    'facet': 'true',
    'facet.range':[
        facet, 
    ],
    'facet.range.start': f'{start}/YEAR',
    'facet.range.end': f'{end}/YEAR',
    'facet.range.gap': '+1YEARS',
}

In [ ]:
solr = pysolr.Solr(url)

In [ ]:
import time
run_start = time.time()

results = solr.search(**solr_params)

run_end = time.time() - run_start
run_end

In [ ]:
# results.facets

In [ ]:
result_counts = results.facets["facet_ranges"][facet]["counts"]
x = result_counts[0::2]
y = result_counts[1::2]


In [ ]:
start = list(map(lambda val: val>0, y)).index(True)
start

In [ ]:
end = list(map(lambda val: val>0, reversed(y))).index(True)
end = len(y) - end
end

In [ ]:
x = x[start:end]
y = y[start:end]

In [ ]:
df = pd.DataFrame(list(zip(x, y)), columns =['date', 'counts'])
df

In [ ]:
#df["date"]=df["date"].str.split(pat="-01-01T", n=1, expand=True)
df["date"]=df["date"].str[:-16]
df["date"]=pd.to_numeric(df["date"], downcast="signed")
df

In [ ]:
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure

source = ColumnDataSource(df)



select = figure(title="Temporal Coverage",
                plot_height=130, plot_width=800, 
                y_axis_type=None,
                tools="", toolbar_location=None, 
                background_fill_color="#efefef")


select.line('date', 'counts', source=source)
select.ygrid.grid_line_color = None

show(select)